In [1]:
push!(LOAD_PATH, "../")
using Integrator
using SpectralGrid
using CubicBSpline
using Chebyshev
using Fourier
using NumericalModels
using Parameters
using CSV
using DataFrames

In [12]:
nodes = 100
lpoints = 0
blpoints = 0
for r = 1:(nodes*3)
    lpoints += 4 + 4*r
    blpoints += 1 + 2*r
end
model = ModelParameters(
    ts = 1.0,
    integration_time = 3600.0,
    output_interval = 3600.0,
    equation_set = "LinearShallowWaterRL",
    initial_conditions = "gauss_RL_ics.csv",
    output_dir = "./linearSW_output/",
    grid_params = GridParameters(xmin = 0.0,
        xmax = 3.0e5,
        num_nodes = nodes,
        rDim = nodes*3,
        b_rDim = nodes+3,
        BCL = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R1T0, 
            "v" => CubicBSpline.R1T0,
            "ub" => CubicBSpline.R1T0, 
            "vb" => CubicBSpline.R1T0,
            "wb" => CubicBSpline.R1T0),
        BCR = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R1T0, 
            "v" => CubicBSpline.R0,
            "ub" => CubicBSpline.R1T0, 
            "vb" => CubicBSpline.R0,
            "wb" => CubicBSpline.R0),
        lDim = lpoints,
        b_lDim = blpoints,
        vars = Dict(
            "h" => 1, 
            "u" => 2, 
            "v" => 3,
            "ub" => 4, 
            "vb" => 5,
            "wb" => 6)))

ModelParameters
  ts: Float64 1.0
  integration_time: Float64 3600.0
  output_interval: Float64 3600.0
  equation_set: String "LinearShallowWaterRL"
  initial_conditions: String "gauss_RL_ics.csv"
  output_dir: String "./linearSW_output/"
  grid_params: GridParameters


In [13]:
grid = createGrid(model.grid_params);

In [38]:
cartesian = getCartesianGridpoints(grid)
gridpoints = getGridpoints(grid)

181800×2 Matrix{Float64}:
  338.105      0.0
  338.105      0.785398
  338.105      1.5708
  338.105      2.35619
  338.105      3.14159
  338.105      3.92699
  338.105      4.71239
  338.105      5.49779
 1500.0        0.261799
 1500.0        0.785398
 1500.0        1.309
 1500.0        1.8326
 1500.0        2.35619
    ⋮          
    2.99662e5  7.00074
    2.99662e5  7.00596
    2.99662e5  7.01118
    2.99662e5  7.0164
    2.99662e5  7.02162
    2.99662e5  7.02683
    2.99662e5  7.03205
    2.99662e5  7.03727
    2.99662e5  7.04249
    2.99662e5  7.04771
    2.99662e5  7.05293
    2.99662e5  7.05815

In [39]:
balanced = CSV.read("/Users/mmbell/Development/scythe/SWslab_output/physical_out_10800.0.csv", DataFrame, header=1)

,r,l,x,y,v,wb,vb,ub
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,338.105,0.0,338.105,0.0,0.33544,-0.388308,0.365151,0.0751673
2,338.105,0.785398,239.076,239.076,0.33544,-0.388308,0.365151,0.0751673
3,338.105,1.5708,2.0703e-14,338.105,0.33544,-0.388308,0.365151,0.0751673
4,338.105,2.35619,-239.076,239.076,0.33544,-0.388308,0.365151,0.0751673
5,338.105,3.14159,-338.105,4.14059e-14,0.33544,-0.388308,0.365151,0.0751673
6,338.105,3.92699,-239.076,-239.076,0.33544,-0.388308,0.365151,0.0751673
7,338.105,4.71239,-6.21089e-14,-338.105,0.33544,-0.388308,0.365151,0.0751673
8,338.105,5.49779,239.076,-239.076,0.33544,-0.388308,0.365151,0.0751673
9,1500.0,0.261799,1448.89,388.229,1.48932,-0.280068,1.61828,0.244768


In [40]:
Rmax = 50000.0
Vmax = 50.0
V0 = Vmax / Rmax
zeta = 2.0 * V0
epsilon = 5000.0
f = 5.0e-5
r_1 = 0.0
h = 0.0
v = 0.0
ubar = 0.0
vbar = 0.0
vprime = 0.0
dhdr_1 = 0.0

# Mean wind
#V = 0.0
#U = -10.0

for i = 1:grid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    x_m = cartesian[i,1]
    y_m = cartesian[i,2]

    # Use the output field for mean vortex
    vfree = balanced.v[i]
    ufree = balanced.u[i]
    h = balanced.h[i]
    vbl = balanced.vb[i]
    ubl = balanced.ub[i]
    wbl = balanced.wb[i]    

    # Add a perturbation
    if (r_m < Rmax)
        #vbar = V0 * r_m
        vprime = 0.5 * zeta * r_m * (epsilon * cos(2 * l_m) / Rmax)
        uprime = 0.5 * zeta * r_m * (epsilon * sin(2 * l_m) / Rmax)
    else
        #vbar = Rmax * Rmax * V0 / r_m
        vprime = 0.5 * zeta * (Rmax^2 / r_m) * (-epsilon * cos(2 * l_m) * Rmax / r_m^2)
        uprime = 0.5 * zeta * (Rmax^2 / r_m) * (epsilon * sin(2 * l_m) * Rmax / r_m^2)
    end
    
    # Calculate balanced h field
    #if r_m > r_1
    #    dhdr = ((f * vbar) + (vbar * vbar / r_m)) / 9.81
    #    h = h + (r_m - r_1) * 0.5 * (dhdr + dhdr_1)
    #    dhdr_1 = dhdr
    #    r_1 = r_m
    #end

    # Gaussian blob test
    #grid.physical[i,1,1] = 100.0*exp(-((x_m-5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2))) + 100.0*exp(-((x_m+5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2)))
    #grid.physical[i,2,1] = (U * cos(l_m)) + (V * sin(l_m))

    grid.physical[i,1,1] = h
    grid.physical[i,2,1] = ufree + uprime
    grid.physical[i,3,1] = vfree + vprime
    grid.physical[i,4,1] = ubl + uprime
    grid.physical[i,5,1] = vbl + vprime
    grid.physical[i,6,1] = wbl
end

In [41]:
h

-0.003997239915712587

In [31]:
#grid.physical[:,1,1] = grid.physical[:,1,1] .- 280.4424824946081

181800-element Vector{Float64}:
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.43636471999883
 -280.3220696505714
 -280.3220696505714
 -280.3220696505714
 -280.3220696505714
 -280.3220696505714
    ⋮
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0

In [42]:
ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=grid.physical[:,1,1], 
    u=grid.physical[:,2,1], 
    v=grid.physical[:,3,1],
    ub = grid.physical[:,4,1],
    vb = grid.physical[:,5,1],
    wb = grid.physical[:,6,1]) 

,r,l,h,u,v,ub,vb,wb
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,338.105,0.0,-280.331,-2.69885e-6,0.36925,0.0751673,0.398961,-0.388308
2,338.105,0.785398,-280.331,0.0338078,0.33544,0.108978,0.365151,-0.388308
3,338.105,1.5708,-280.331,-2.69885e-6,0.301629,0.0751673,0.33134,-0.388308
4,338.105,2.35619,-280.331,-0.0338132,0.33544,0.0413568,0.365151,-0.388308
5,338.105,3.14159,-280.331,-2.69885e-6,0.36925,0.0751673,0.398961,-0.388308
6,338.105,3.92699,-280.331,0.0338078,0.33544,0.108978,0.365151,-0.388308
7,338.105,4.71239,-280.331,-2.69885e-6,0.301629,0.0751673,0.33134,-0.388308
8,338.105,5.49779,-280.331,-0.0338132,0.33544,0.0413568,0.365151,-0.388308
9,1500.0,0.261799,-280.218,0.0749919,1.61923,0.319768,1.74819,-0.280068


In [43]:
CSV.write("../SWslab_wave2_ics.csv",ics)

"../SWslab_wave2_ics.csv"